Importing Packages 

In [1]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns  
# You can configure the format of the images: ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’.
%config InlineBackend.figure_format = 'svg'
# this statement allows the visuals to render within your Jupyter Notebook
%matplotlib inline 

from sklearn.model_selection import train_test_split, KFold 
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import patsy
import scipy.stats as stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

Functions

In [14]:
def get_tts_dfs(df, y_name):
    X, y = df.drop([y_name],axis=1), df[y_name] 
    xX_train, xX_test, yy_train, yy_test = train_test_split(X, y, test_size=.2, random_state=10)
    xX_train, yy_train = np.array(xX_train), np.array(yy_train)
    return xX_train, xX_test, yy_train, yy_test

def scaler(df_to_fit_transform,df_to_transform):
    scaler = StandardScaler()
    df_to_fit_transform_scaled = scaler.fit_transform(df_to_fit_transform)
    df_to_transform_scaled = scaler.transform(df_to_transform)
    return df_to_fit_transform_scaled, df_to_transform_scaled

def crossvalidate(df, y_name, r_alpha, l_alpha):
    X_train, X_test, y_train, y_test = get_tts_dfs(df,y_name)
    
    X_train, y_train = np.array(X_train), np.array(y_train) 
    
    kf = KFold(n_splits=5, shuffle=True, random_state = 71)
    cv_lm_r2s, cv_lm_poly_r2s, cv_lm_reg_r2s, cv_lm_lasso_r2s = [], [], [], []#collect the validation results for both models

    for train_ind, val_ind in kf.split(X_train,y_train):
        X_trainfold, y_trainfold = X_train[train_ind], y_train[train_ind]
        X_val, y_val = X_train[val_ind], y_train[val_ind] 
        
        # standardising
        X_trainfold_scaled, X_val_scaled = scaler(X_trainfold, X_val)

        #simple linear regression  
        lm = LinearRegression()
        lm.fit(X_trainfold_scaled, y_trainfold)
        cv_lm_r2s.append(lm.score(X_val_scaled, y_val))

        #polynomial with feature scaling
        lm_poly = LinearRegression()
        poly = PolynomialFeatures(degree=2) 
        X_train_poly = poly.fit_transform(X_trainfold_scaled)
        X_val_poly = poly.transform(X_val_scaled)
        lm_poly.fit(X_train_poly, y_trainfold)
        cv_lm_poly_r2s.append(lm_poly.score(X_val_poly, y_val))

        #ridge regression with feature scaling 
        lm_reg = Ridge(alpha = r_alpha)
        lm_reg.fit(X_trainfold_scaled, y_trainfold)
        cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

        #lasso regression with feature scaling 
        lm_lasso = Lasso(alpha = l_alpha)
        lm_lasso.fit(X_trainfold_scaled, y_trainfold)
        cv_lm_lasso_r2s.append(lm_lasso.score(X_val_scaled, y_val))

    return print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f} \n'
                 f'Polynomial mean cv r^2: {np.mean(cv_lm_poly_r2s):.3f} +- {np.std(cv_lm_poly_r2s):.3f}\n'
                 f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}\n'
                 f'Lasso mean cv r^2: {np.mean(cv_lm_lasso_r2s):.3f} +- {np.std(cv_lm_lasso_r2s):.3f}')

In [19]:
crossvalidate(df, 'log_dengue_cases', 1, 1)

Simple mean cv r^2: 0.755 +- 0.041 
Polynomial mean cv r^2: 0.441 +- 0.153
Ridge mean cv r^2: 0.755 +- 0.043
Lasso mean cv r^2: -0.010 +- 0.012


Opening pickle from cleaning notebook

In [6]:
with open('//Users/adelweiss/Documents/Project/Project 2/data/df2.pickle','rb') as read_file:
    df = pickle.load(read_file)

Cleaning combined dataset 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248 entries, 50 to 210
Data columns (total 16 columns):
dengue_cases       248 non-null float64
eweek              248 non-null object
rainfall           248 non-null float64
mean_temp          248 non-null float64
max_temp           248 non-null float64
min_temp           248 non-null float64
fever              248 non-null float64
nausea             248 non-null float64
headache           248 non-null float64
ache_pain          248 non-null float64
eye_pain           248 non-null float64
dengue             248 non-null float64
rashes             248 non-null float64
vomitting          248 non-null float64
dengue_cases_+1    248 non-null float64
temp_range         248 non-null float64
dtypes: float64(15), object(1)
memory usage: 32.9+ KB


In [8]:
### checking out the histo gram 
#df_pairplot = sns.pairplot(df)
#df_pairplot.savefig('pairplot.svg')

In [9]:
### log transform dengue cases to correct for postive skew 
### I tried the other transformation as well, but log seems to have the best results 

df['log_dengue_cases'] = np.log(df['dengue_cases_+1'])
#df['sq root Dengue Cases Week+1'] = np.sqrt(df['Dengue Cases Week+1'])
#df['cbrt Dengue Cases Week+1'] = np.cbrt(df['Dengue Cases Week+1'])

In [10]:
### correlation between dengue and dengue cases improved after log transformation
df['log_dengue'] = np.log(df['dengue'])
### but the correlation didn't improve for other variables 
df.corr()

,dengue_cases,rainfall,mean_temp,max_temp,min_temp,fever,nausea,headache,ache_pain,eye_pain,dengue,rashes,vomitting,dengue_cases_+1,temp_range,log_dengue_cases,log_dengue
dengue_cases,1.000000,-0.071893,0.210822,0.075165,0.187541,0.385912,-0.072115,-0.315066,-0.339293,0.035659,0.883026,0.184837,-0.235535,0.962818,-0.114570,0.889229,0.867830
rainfall,-0.071893,1.000000,-0.414244,-0.371848,-0.515243,0.085435,0.028210,0.101248,0.027212,0.015310,-0.043308,0.113566,0.153593,-0.066101,0.118183,-0.099180,-0.049353
mean_temp,0.210822,-0.414244,1.000000,0.782483,0.905729,0.040537,0.008866,-0.095343,-0.031297,0.034176,0.214145,-0.104796,-0.161695,0.212411,-0.054584,0.244784,0.237247
max_temp,0.075165,-0.371848,0.782483,1.000000,0.617259,-0.099652,0.038033,-0.048595,-0.066429,-0.011313,0.082671,-0.199685,-0.176929,0.084623,0.517724,0.156644,0.117375
min_temp,0.187541,-0.515243,0.905729,0.617259,1.000000,0.094833,-0.004644,-0.112109,-0.003192,0.032512,0.176691,-0.079106,-0.130971,0.187859,-0.353540,0.220650,0.190191
fever,0.385912,0.085435,0.040537,-0.099652,0.094833,1.000000,0.089924,0.115459,0.085456,0.092562,0.474546,0.151737,0.069081,0.387706,-0.221606,0.260463,0.402009
nausea,-0.072115,0.028210,0.008866,0.038033,-0.004644,0.089924,1.000000,0.051616,0.151909,0.015444,-0.051263,0.017301,0.104073,-0.078207,0.050270,-0.063869,-0.081546
headache,-0.315066,0.101248,-0.095343,-0.048595,-0.112109,0.115459,0.051616,1.000000,0.390927,-0.051586,-0.207473,0.020063,0.199242,-0.321138,0.064134,-0.430801,-0.256573
ache_pain,-0.339293,0.027212,-0.031297,-0.066429,-0.003192,0.085456,0.151909,0.390927,1.000000,0.087258,-0.208140,-0.019998,0.276847,-0.337107,-0.075509,-0.475114,-0.297927
eye_pain,0.035659,0.015310,0.034176,-0.011313,0.032512,0.092562,0.015444,-0.051586,0.087258,1.000000,0.017386,-0.043163,0.097802,0.037519,-0.048805,0.028928,-0.004672


In [16]:
df_assumptions = df.copy() ### keeping a copy for assumption testing later 

In [17]:
### making sure that these variables are not in the model
### removing transformed variables, and min_temp because it is highly correlated with mean_temp
df.drop(columns = ['dengue_cases','eweek','dengue_cases_+1','dengue','min_temp'], inplace = True)

In [18]:
df

,rainfall,mean_temp,max_temp,fever,nausea,headache,ache_pain,eye_pain,rashes,vomitting,temp_range,log_dengue_cases,log_dengue
50,1.628571,27.485714,31.357143,77.0,77.0,76.0,84.0,61.0,60.0,96.0,5.914286,5.075174,3.332205
49,7.000000,27.457143,30.842857,67.0,92.0,56.0,87.0,81.0,58.0,86.0,5.957143,4.844187,3.258097
48,7.314286,27.157143,31.128571,82.0,70.0,70.0,77.0,59.0,54.0,89.0,7.171429,4.672829,3.367296
47,5.914286,27.228571,31.114286,76.0,64.0,80.0,82.0,76.0,62.0,78.0,6.771429,4.727388,3.295837
46,3.800000,27.457143,31.600000,67.0,56.0,70.0,82.0,72.0,67.0,73.0,6.585714,4.691348,2.995732
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,0.000000,26.071429,30.371429,58.0,71.0,78.0,80.0,37.0,53.0,62.0,7.028571,5.262690,3.871201
213,0.000000,25.757143,29.014286,64.0,67.0,65.0,80.0,43.0,58.0,91.0,5.371429,5.910797,3.737670
212,2.600000,26.271429,29.100000,66.0,67.0,69.0,73.0,43.0,84.0,65.0,4.742857,5.609472,3.526361
211,8.171429,26.171429,29.614286,72.0,67.0,56.0,75.0,48.0,78.0,68.0,5.557143,5.455321,3.850148


Cross validation of regressions model 

In [20]:
###split into train and test 
X, y = df.drop(['log_dengue_cases'],axis=1), df['log_dengue_cases']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)
X_train, y_train = np.array(X_train), np.array(y_train) 

In [21]:
### then compare which is the best models...
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_poly_r2s, cv_lm_reg_r2s, cv_lm_lasso_r2s = [], [], [], []#collect the validation results for both models

for train_ind, val_ind in kf.split(X_train,y_train):
    X_trainfold, y_trainfold = X_train[train_ind], y_train[train_ind]
    X_val, y_val = X_train[val_ind], y_train[val_ind] 
    
    #Scale variables 
    scaler = StandardScaler()
    X_trainfold_scaled = scaler.fit_transform(X_trainfold)
    X_val_scaled = scaler.transform(X_val)
    
    #simple linear regression  
    lm = LinearRegression()
    lm.fit(X_trainfold_scaled, y_trainfold)
    cv_lm_r2s.append(lm.score(X_val_scaled, y_val))
    
    #polynomial with feature scaling
    lm_poly = LinearRegression()
    poly = PolynomialFeatures(degree=2) 
    X_train_poly = poly.fit_transform(X_trainfold_scaled)
    X_val_poly = poly.transform(X_val_scaled)
    lm_poly.fit(X_train_poly, y_trainfold)
    cv_lm_poly_r2s.append(lm_poly.score(X_val_poly, y_val))
    
    #ridge regression with feature scaling 
    lm_reg = Ridge()
    #lm_reg = Ridge(alpha=14)
    lm_reg.fit(X_trainfold_scaled, y_trainfold)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))
    
    #lasso regression with feature scaling 
    lm_lasso = Lasso()
    #lm_lasso = Lasso(alpha = 0.01)
    lm_lasso.fit(X_trainfold_scaled, y_trainfold)
    cv_lm_lasso_r2s.append(lm_lasso.score(X_val_scaled, y_val))
    
print('Simple regression scores: ', cv_lm_r2s)
print('Polynomial scores: ', cv_lm_poly_r2s, '\n')
print('Ridge scores: ', cv_lm_reg_r2s, '\n')
print('Lasso scores: ', cv_lm_lasso_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Polynomial mean cv r^2: {np.mean(cv_lm_poly_r2s):.3f} +- {np.std(cv_lm_poly_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')
print(f'Lasso mean cv r^2: {np.mean(cv_lm_lasso_r2s):.3f} +- {np.std(cv_lm_lasso_r2s):.3f}')

Simple regression scores:  [0.701782356229927, 0.795835525776602, 0.7198582333294903, 0.8082210815459658, 0.7495764360743337]
Polynomial scores:  [0.5868374590623486, 0.4952312375337607, 0.18689883294838308, 0.5853645041913444, 0.3508085748764742] 

Ridge scores:  [0.7021045489718941, 0.7977723571577625, 0.7174782213510572, 0.8106409878098457, 0.7482056743569476] 

Lasso scores:  [-4.493844615005571e-05, -7.808818365262482e-07, -0.02852945255842787, -0.0012294067577502954, -0.02211139359063341] 

Simple mean cv r^2: 0.755 +- 0.041
Polynomial mean cv r^2: 0.441 +- 0.153
Ridge mean cv r^2: 0.755 +- 0.043
Lasso mean cv r^2: -0.010 +- 0.012


Looks like simple linear is the way to go. Let's try to do some feature selection using Lasso

In [22]:
### looks like non-polynomial is the way to go.... 
### let's try to reduce the model complexity 
### thru feature selection before we optimise alpha for ridge

#need to redo this cuz CV does not do well with arrays 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)
### Scaling the variables 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) 

### Applying lasso model on the train dataset 
alphavec = 10**np.linspace(-2,2,200)
lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_train_scaled, y_train)
list(zip(X_train, lasso_model.coef_))

[('rainfall', -0.0),
 ('mean_temp', 0.0),
 ('max_temp', 0.06780687766362478),
 ('fever', 0.0),
 ('nausea', 0.011158723779936918),
 ('headache', -0.14677397514147342),
 ('ache_pain', -0.16583804313277145),
 ('eye_pain', 0.02527453506350347),
 ('rashes', 0.05901780751365415),
 ('vomitting', -0.034139794592951786),
 ('temp_range', -0.021198328921619804),
 ('log_dengue', 0.5761032660682472)]

In [23]:
###dropping columns that hit zero or v close to zero for optimal lasso 
df = df.drop(columns = ['rainfall','mean_temp','fever'])

In [24]:
### let's see if ridge does better after optimization 
X, y = df.drop(['log_dengue_cases'],axis=1), df['log_dengue_cases']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)
### Scaling the variables 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

### Applying ridge model on the train dataset 
alphavec = 10**np.linspace(-2,2,200)
reg_model = RidgeCV(alphas = alphavec, cv=5)
reg_model.fit(X_train_scaled, y_train)
optimal_alpha = reg_model.alpha_
reg_model.alpha_


//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


8.21434358491943

In [25]:
X_train, y_train = np.array(X_train), np.array(y_train) 

In [26]:
### then compare which is the best models...
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s  = [], []#collect the validation results for both models

for train_ind, val_ind in kf.split(X_train,y_train):
    X_trainfold, y_trainfold = X_train[train_ind], y_train[train_ind]
    X_val, y_val = X_train[val_ind], y_train[val_ind] 
    
    #Scale variables 
    scaler = StandardScaler()
    X_trainfold_scaled = scaler.fit_transform(X_trainfold)
    X_val_scaled = scaler.transform(X_val)
    
    #simple linear regression using feature scaling 
    lm = LinearRegression()
    lm.fit(X_trainfold_scaled, y_trainfold)
    cv_lm_r2s.append(lm.score(X_val_scaled, y_val))
        
    #ridge regression with feature scaling 
    lm_reg = Ridge(alpha = optimal_alpha)
    lm_reg.fit(X_trainfold_scaled, y_trainfold)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))
    
print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')


Simple regression scores:  [0.7060926714798297, 0.8027074102784173, 0.715096509087561, 0.8113438012852092, 0.7508956454425376]
Ridge scores:  [0.6989345920006113, 0.8009208626534889, 0.7127646636029199, 0.8159409378348983, 0.7576801227997284] 

Simple mean cv r^2: 0.757 +- 0.043
Ridge mean cv r^2: 0.757 +- 0.046


In [ ]:
### Let's stick to the linear model 

Model Evaluation 

In [ ]:
### let's see if ridge does better after optimization 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)
### Scaling the variables 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

In [ ]:
df_for_eval = pd.DataFrame(X_train_scaled)
df_for_eval.columns = list(X_train.columns)

In [ ]:
y_train_df = pd.DataFrame(y_train).reset_index().drop(columns = ['index'])
df_for_eval['log_dengue_cases'] = y_train_df

In [ ]:
df_for_eval.info()

In [ ]:
# Create your feature matrix (X) and target vector (y)
y, X = patsy.dmatrices('log_dengue_cases ~ max_temp + nausea + headache + ache_pain + eye_pain + rashes + vomitting + temp_range + log_dengue', 
                       data=df_for_eval, return_type="dataframe")

# Create your model
model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

In [ ]:
### remove nausea - no change in r2 or adj r2
### remove temp_range - r2 dropped by .002, adj r2 same 
### r2s drop after subsequent removal of variables 
# Create your feature matrix (X) and target vector (y)
y, X = patsy.dmatrices('log_dengue_cases ~ max_temp + headache + ache_pain + eye_pain + rashes + vomitting + log_dengue', 
                       data=df_for_eval, return_type="dataframe")



# Create your model
model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

In [ ]:
### let's see of this simplier model fairs better 

df.drop(columns = ['nausea', 'temp_range'], inplace = True)
X, y = df.drop(['log_dengue_cases'],axis=1), df['log_dengue_cases']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) 

In [ ]:
lm = LinearRegression()
lm.fit(X_train_scaled, y_train)
lm.score(X_test_scaled, y_test)
fit = lm.fit(X_train_scaled, y_train)
r_test = lm.score(X_test_scaled, y_test)

In [ ]:
r_test

In [ ]:
adjusted_r2 = 1-(1-r_test)*(X_train_scaled.shape[0]-1)/(X_train_scaled.shape[0]-X_train_scaled.shape[1]-1)

In [ ]:
adjusted_r2

Checking Model Assumptions 

In [ ]:
### check if residuals are normally distributed 
data = pd.DataFrame()
data['predict']=fit.predict(X_train_scaled)
data['y_train'] = pd.DataFrame(y_train).reset_index().drop(columns = ['index'])
data['resid']=data.y_train-data.predict
with sns.axes_style('white'):
    plot=data.plot(kind='scatter',
                  x='predict',y='resid',alpha=0.2,figsize=(10,6))

In [ ]:
data.y_train.hist()

In [ ]:
stats.probplot(data['resid'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

In [ ]:
lamb=stats.boxcox_normmax(data.y_train, brack=(-1.9, 1.9)) # don't use "lambda" as it's a Python reserved word
print("Lambda:", lamb)
y_t=(np.power(data.y_train,-lamb)-1)/-lamb

plt.hist(y_t);

Check that errors are uncorrelated across observations 

In [ ]:
with open('//Users/adelweiss/Documents/Project/Project 2/data/df2.pickle','rb') as read_file:
    df = pickle.load(read_file)

df

In [ ]:
df_tocheckassumptions.drop(columns = ['Dengue Cases','Dengue Cases Week+1'], inplace = True)
df_tocheckassumptions.columns = ['eweek','rainfall', 'mean_temp', 'max_temp', 'min_temp', 'fever', 'nausea', 'headache',
             'ache_pain', 'eye_pain', 'dengue', 'rashes', 'vomitting', 'temp_range', 'log_dengue_cases']

In [ ]:
df_tocheckassumptions.drop(columns = ['rainfall','mean_temp','fever','nausea','eye_pain', 
                        'vomitting', 'temp_range', 'min_temp'], inplace = True)

In [ ]:
X, y = df_tocheckassumptions.drop(['log_dengue_cases'],axis=1), df_tocheckassumptions['log_dengue_cases']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) 

In [ ]:
X_train

In [ ]:
y_train['resid'] = fit.predict(X_train_scaled)